# 6.4 Vector Store

이번에는 이전에 split했던 문서를 embed 할 것이다. 그전에 먼저 embedding model에 대해 살펴보자

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

print(embedder.embed_query("Hi")) # query를 임베드하는 메서드인 embed_query
print(len(embedder.embed_query("Hi")))


[-0.03629858192333016, -0.007224538187570183, -0.033718855541097256, -0.028663632678071895, -0.026865641732513677, 0.03460482274185761, -0.012318847263635711, -0.007752209747023989, 0.0019380524367559973, -0.0027018730680822924, 0.024781013901381176, -0.0024771241998875156, -0.005732726535614379, -0.0029054499465086627, 0.0066773232887656405, -0.0030324821179497563, 0.03384914384922042, -0.0015032120884641694, 0.021093827586875214, -0.008996472123429593, -0.02171921630874401, 0.010384052476961034, 0.0062441115908914826, 0.007081220210444346, -0.01231233266196503, 0.0008998100308185957, 0.005876044512740216, -0.009888952994538019, -0.0030731974470689, -0.024572550373209837, 0.010742348118267585, -0.013810659381252822, -0.024429232861745306, -0.014110324538845857, 0.0024347802203507018, -0.018878911447619544, 0.000561872345109932, -0.01127001874639878, 0.018110203351640992, -0.009967126351940966, 0.013028923944578134, -0.011328649230112295, -0.009133275964546056, -0.009654432922329181, -

위의 출력값이 'Hi'에 해당하는 벡터값이고, 1536이 차원의 갯수이다. 이제 문서를 embed해보자.

In [9]:
embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "Hi",
    "how",
    "are",
    "you",
    "longer sentences" # 각 요소가 꼭 하나의 단어일 필요는 없다.
])
# embed_documents는 여러개의 string의 배열로 되어있어야 한다.

print(vector)
print(len(vector))

[[-0.03629858192333016, -0.007224538187570183, -0.033718855541097256, -0.028663632678071895, -0.026865641732513677, 0.03460482274185761, -0.012318847263635711, -0.007752209747023989, 0.0019380524367559973, -0.0027018730680822924, 0.024781013901381176, -0.0024771241998875156, -0.005732726535614379, -0.0029054499465086627, 0.0066773232887656405, -0.0030324821179497563, 0.03384914384922042, -0.0015032120884641694, 0.021093827586875214, -0.008996472123429593, -0.02171921630874401, 0.010384052476961034, 0.0062441115908914826, 0.007081220210444346, -0.01231233266196503, 0.0008998100308185957, 0.005876044512740216, -0.009888952994538019, -0.0030731974470689, -0.024572550373209837, 0.010742348118267585, -0.013810659381252822, -0.024429232861745306, -0.014110324538845857, 0.0024347802203507018, -0.018878911447619544, 0.000561872345109932, -0.01127001874639878, 0.018110203351640992, -0.009967126351940966, 0.013028923944578134, -0.011328649230112295, -0.009133275964546056, -0.009654432922329181, 

In [10]:
print(len(vector[0]), len(vector[1]), len(vector[2]))

1536 1536 1536


각 단어의 벡터의 차원은 다 같다는것을 알 수 있다.

이제 우리의 문서를 embed 해보자.

우리는 코드를 실행할 때마다 embed를 하지 않을것이다. 왜냐하면 임베드 하는데는 돈이 들기 때문이다. 대신에 우리는 한번만 임베드 하여서 그 결과를 저장할 것이다.<br>
여기서 vector store라는것이 등장한다. vector store에 대해 살펴보자.<br>

vector store는 일종의 데이터 베이스라고 생각하면 된다. 벡터 공간에서 검색도 할 수 있게 해준다. 그러니까 우리가 벡터들을 만들고 나서, 그것들을 캐시해주고, vector store에 그 벡터들을 넣어주면, 우리가 검색을 할 수 있다. 관련있는 문서들만 찾아낼 수 있게 되는것이다.

LangChain은 다양한 Vector store들을 지원하고 있다. 그 중 일부는 cloud에 있는데, 어떤건 유료지만 어떤건 무료로 사용할 수 있다. 지금은 오픈소스를 사용해 볼 것이다. 클라우드 환경이 아니라 우리의 컴퓨터에서 직접 실행될 것이고, 이름은 Chroma이다.<br>
Chroma는 아주 잘 작동하는 도구이기 때문에, 우리는 이것을 사용할 것이다. Chroma를 이용해서 우리의 문서들을 계산해서 얻어낸 vector들 속에서 검색도 할 수 있다.

연습해보자!!

In [12]:
from langchain.vectorstores import Chroma

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(docs, embeddings)
# 이렇게 from_documents라는 클래스 메서드로 클래스를 생성해주고,
# split된 문서와 OpenAI 임베딩 모델을 전달해 주어야 한다.

# 이 코드에 에러가 없다면 코드를 실행하는 순간, 요금이 나가게 된다.

In [15]:
results = vectorstore.similarity_search("What are the contents of this document?")

print(len(results))

print("\n")
print(results)

4


[Document(page_content='“Honored guests,” he began, his voice carrying an almost musical resonance, “welcome to the Grand Masquerade, where the borders between the impossible and the tangible blur. Tonight, we celebrate the sublime. Marvels of invention! Feats of magic! Curiosities beyond mortal comprehension!” He spread his arms wide. “I am Lord Lucien Vale, your humble host.”\nA ripple of admiration and excitement swept through the guests, each exchanging glances behind their disguises. \nLord Lucien continued, “We shall open this night of wonders with our first unveiling. We have heard rumors of a clockmaker who has mastered the impossible. Step forward, Oldwyn Rook.” \nStunned, Oldwyn felt hundreds of masked faces pivoting to stare at him. How had Lord Lucien known his name already? Heart pounding, Oldwyn swallowed hard and ascended the platform steps. The hush of anticipation pressed in on him like a physical weight.\n“Show us your creation,” Lord Lucien intoned.\nWith trembli

잘 출력이 되는것을 알 수 있다.<br>
이것으로 우리는 왜 텍스트를 작은 덩어리로 분해하는것이 좋은지 깨달았다. 위의 예시처럼 우리가 vector store, vector data base에서 질문(query)를 검색하면 그 질문(query)과 관련있는 문서들을 반환받게 된다. 만약 받은 문서들이 너무 크다면, 엄청 큰 문서들이 LLM에게 전달될거고, 그만큼 큰 돈을 지출하게 될것이다. 그러니까 작은 부분으로 분할(split)해주는것은 좋은 아이디어이다. 물론 모든 문맥과 의미가 소멸할만큼 작은것은 안좋다.

이제 우리는 이 Embeddings을 캐싱해줄 것이다. 왜냐하면, 다시 실행하면 이것들은 사라지고, 재실행하면 다시 계산해야 된다. 그러면 지출도 그만큼 늘게 된다. 그렇게 되는것을 막기 위해 캐싱을 해줄 것이다.

여기서 캐싱이란 데이터를 임시로 저장해두는 방법이다. 쉽게 설명하면 다음과 같다.

1. 데이터 저장
    - 한 번 계산하거나 생성한 데이터를 저장해두는 것
    - 나중에 같은 데이터가 필요할 때 다시 계산하지 않고 저장해둔 것을 사용
2. 시간과 비용 절약
    - 매번 같은 작업을 반복하지 않아도 됨
    - 특히 OpenAI API처럼 비용이 발생하는 경우 매우 중요

In [17]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache")
# 여기에 embed된 embeddings가 저장될 것이다.

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
     embeddings,
     cache_dir,
)
# 이 from_bytes_store메서드는 embedding작업을 위해 필요한 embedder의 입력을 요구한다.
# 또한 embeddings을 저장할 장소인 캐시도 전달해 줘야한다.

vectorstore = Chroma.from_documents(docs, cached_embeddings)
# 이제 우리가 from_documents를 호출하면 OpenAIEmbeddings대신 미리 cache되어있는 embeddings를 전달한다.